In [9]:
import numpy as np
from collections import defaultdict

In [10]:
def bin2int(arr):
    y = 0
    for i, j in enumerate(arr):
        y += j<<i
    return y


In [11]:
with open("config.txt", "r") as config_file:
    n, l = map(int, config_file.readline().split())
    h = np.array(list(map(float, config_file.readline().split())))
    sigma_squared = float(config_file.readline()) 
    sigma = np.sqrt(sigma_squared)

In [12]:
print("n: ",n,"\nl: ",l)
print("Weights: ", h)
print("Sigma: ",sigma)

n:  2 
l:  2
Weights:  [0.7 0.5]
Sigma:  0.4743416490252569


In [13]:
with open("train.txt") as train_file:
    train_bits = list(map(int, train_file.readline()))

In [14]:
train_bits[:10]

[1, 1, 0, 0, 1, 0, 0, 0, 0, 0]

In [15]:
samples = []
for i in range(len(train_bits)-n+1):
    # print(train_bits[i:i+n])
    samples.append(train_bits[i:i+n])
samples = np.array(samples)

In [16]:
samples = np.array(samples)
I = samples.dot(h)
for i in range(len(I)):
    I[i] += np.random.normal(loc=0.0, scale=sigma)

In [17]:
prior_probabilities = defaultdict(float)
transition_probabilities = np.zeros((2**n, 2**n))
clusters = defaultdict(list)
cluster_means = defaultdict(float)

In [18]:
clusters[bin2int(samples[0])].append(I[0])
prior_probabilities[bin2int(samples[0])] += 1
for i in range(1, len(I)):
    prior_probabilities[bin2int(samples[i])] += 1
    transition_probabilities[bin2int(samples[i])][bin2int(samples[i - 1])] += 1
    clusters[bin2int(samples[i])].append(I[i])

print("\n\n\nTransition count:")
print(transition_probabilities)

transition_probabilities /= transition_probabilities.sum(axis=1, keepdims=True)

for i in range(2**n):
    cluster_means[i] = (np.average(clusters[i]))

for i in range(2**n):
    prior_probabilities[i] /= len(I)





Transition count:
[[1250118. 1249934.       0.       0.]
 [      0.       0. 1249700. 1250095.]
 [1249934. 1249861.       0.       0.]
 [      0.       0. 1250094. 1250261.]]


In [19]:
print("\n\n\nPrior probabilities:")
for i in range(2**n):
    print(i, "\t :\t",  round(prior_probabilities[i], 2))
# print(prior_probabilities)

print("\n\n\nTransition probabilities:")
for i in range(2**n):
    print(i, "\t : ", end="")
    for j in range(2**n):
        print("\t%.2f" % (transition_probabilities[i][j]), end="")
    print("")
# print(transition_probabilities)

# print(clusters)

print("\n\n\nObservation means:")
for i in range(2**n):
    print(i, "\t : \t%.2f" %(cluster_means[i]))
# print(cluster_means)




Prior probabilities:
0 	 :	 0.25
1 	 :	 0.25
2 	 :	 0.25
3 	 :	 0.25



Transition probabilities:
0 	 : 	0.50	0.50	0.00	0.00
1 	 : 	0.00	0.00	0.50	0.50
2 	 : 	0.50	0.50	0.00	0.00
3 	 : 	0.00	0.00	0.50	0.50



Observation means:
0 	 : 	0.00
1 	 : 	0.70
2 	 : 	0.50
3 	 : 	1.20


In [21]:
with open("test.txt") as test_file:
    test_bits = list(map(int, test_file.readline()))

res = []
test_samples = []
for i in range(len(test_bits)-n+1):
    test_samples.append(test_bits[i:i+n])

test_samples = np.array(test_samples)
X = test_samples.dot(h)
for i in range(len(X)):
    X[i] += np.random.normal(loc=0.0, scale=sigma) 